In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [2]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_18"
datedir = "230120"
sttime = "2022-04-01 23:00:00"

In [3]:
#解析結果の読み込み
timeseries_sep = np.load("./" + datedir+'/'+ npyname + ".npy")

In [4]:
timeseries_sep.shape

(24, 207)

In [5]:
timeseries_sep

array([[ 88,  62,  95, ..., 109, 109,   1],
       [ 77,  74,  90, ..., 116, 116,   0],
       [ 64,  74,  86, ..., 106, 106,   0],
       ...,
       [ 13,   4,  24, ...,  26,  26,   0],
       [ 21,  12,  12, ...,  28,  28,   0],
       [ 16,   6,  11, ...,  23,  23,   0]], dtype=int64)

In [6]:
#メッシュ情報の読み込み
csv_file = r'./'+datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["gridcode"] #gridcode #end_mesh_area

In [7]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

for lp in range(4):
    cur =connection.cursor()
    curs =connection.cursor()

    checkMeshtbranList = [] #確認対象メッシュのKDDI値

    #まずは時刻ごとのメッシュ一覧を取得
    sql = "select mesh from " + dbname + " WHERE times = '"+str(dte)+"'"
    cur.execute(sql)
    meshLists = cur.fetchall()
    
    print("time =" + str(dte) + " target =" + str(len(meshLists)))
    for mesh in meshLists:
        
        meshkey = str(int(mesh[0]))
        
        #トラカン値を持つメッシュならば解析しない
        find = 0
        for chkm in df_i.values:
            if str(int(chkm)) == meshkey:
                find = 1
                break
        if find == 1:
            continue

        sql = "select kddi_tran.tbran, T.gridcode,T.断面交通量,kddi_tran.mesh "
        sql += "from " + dbname + " AS kddi_tran "
        sql += "INNER JOIN pointdata as a ON kddi_tran.mesh = cast(a.grid_code AS BIGINT) "
        sql += "CROSS JOIN LATERAL "
        sql += "("
        sql += "SELECT 時刻, gridcode , 断面交通量 from trakan2204_z3out AS trakan_tran "
        sql += "inner JOIN pointdata as b ON trakan_tran.gridcode = cast(grid_code AS BIGINT) "
        sql += "WHERE 時刻 = '"+str(dte)+"' "
        sql += "ORDER BY a.geom <-> b.geom, ST_Distance(a.geom, b.geom) "
        sql += "LIMIT 1) AS T "
        sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + meshkey
        curs.execute(sql)
        datalists = curs.fetchall()
        #print(datalists) #(88.0, 50324526133, 13) #対象MESH値、確認対象メッシュ、対象の断面交通量
        
        if len(datalists) == 0:
            #print(sql)
            continue

        print(meshkey)
        #確認対象メッシュの kddi_tran.tbran値
        enabledcheck = meshkey in checkMeshtbranList
        baseMeshItem = 0
        if enabledcheck == True:
            for meshitem in checkMeshtbranList:
                if str(int(mesh)) == meshitem[0]:
                    baseMeshItem = meshitem[1]
                    break
        else:
            cur3 =connection.cursor()
            sql = "select kddi_tran.tbran from " + dbname + " AS kddi_tran "
            sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + str(datalists[0][1])
            cur3.execute(sql)
            baseMeshParam = cur3.fetchall()
            setparam = 0
            if len(baseMeshParam) != 0:
                setparam = baseMeshParam[0][0]
            if len(checkMeshtbranList) == 0:
                checkMeshtbranList = [meshkey , setparam]
            else:
                checkMeshtbranList.append([meshkey , setparam])
            cur3.close()
            baseMeshItem = setparam

        #メッシュ値を算出
        if baseMeshItem == 0:
            param = 0
        else:
            param = datalists[0][0] / baseMeshItem * datalists[0][2]
        print(str(datalists[0][0]) + ":" +  str(baseMeshItem) + ":" +  str( datalists[0][2]) )
            
        sql = "UPDATE " + dbname + " set tbran = "+ str(param) + " where times = '"  + str(dte) + "' and mesh = " + meshkey + ";"
        #print(sql)
        cur.execute(sql)
        connection.commit()
    cur.close()
    curs.close()
    dte = dte + datetime.timedelta(minutes=15)
connection.commit()

time =2022-04-01 23:00:00 target =6509
50327682241
0.012225824:18.0:6
50327682244
0.003621925:18.0:6
50327682312
0.0020567267:18.0:6
50327682313
0.0013335444:18.0:6
50327682321
0.0005806852:18.0:6
50327682322
0.016126852:18.0:6
50327683121
0.021473378:18.0:6
50327683122
0.0005025063:18.0:6
50327683133
0.00030587125:18.0:6
50327683134
0.004748881:18.0:6
50327683211
0.0017678263:18.0:6
50327683212
0.00018697046:18.0:6
50327683221
0.007614471:18.0:6
50327683421
1.1976692e-05:18.0:6
50327687143
0.00043737254:27.0:9
50327687233
0.0035777192:27.0:9
50327687413
0.00061071594:27.0:9
50327691243
0.009187643:18.0:6
50327691414
0.00096152816:18.0:6
50327691421
0.036896218:18.0:6
50327691422
0.00041395798:18.0:6
50327691423
0.0012374937:18.0:6
50327691432
0.00095504615:18.0:6
50327691433
0.0005451202:18.0:6
50327691444
0.005928057:18.0:6
50327693133
0.0011384133:18.0:6
50327693323
0.00643645:27.0:9
50327700222
0.0021174857:36.0:7
51320602322
0.0026441189:27.0:9
51320602324
3.4041354e-05:27.0:9
513

0.004413198:27.0:22
50325527344
0.0016787769:27.0:22
50325527412
0.0070389286:27.0:22
50325527422
0.0004573865:27.0:22
50325527423
0.0012741499:27.0:22
50325527424
0.004142534:27.0:22
50325528111
0.0020132153:27.0:22
50325528322
0.00029616372:27.0:22
50325528412
0.000344133:27.0:22
50325528413
0.00047802832:27.0:22
50325528414
0.0065452023:27.0:22
50325529214
0.008048387:76.0:9
50325529224
0.0006786962:76.0:9
50325529242
0.007101998:76.0:9
50325529243
0.0022594803:76.0:9
50325529434
0.016672147:76.0:9
50325529443
0.010886528:76.0:9
50325529444
0.000885922:76.0:9
50325535222
0.0012987682:27.0:22
50325535242
0.002405507:136.0:12
50325536142
0.0055746296:136.0:12
50325536143
0.008946364:136.0:12
50325536211
0.003630545:27.0:22
50325536212
0.010951834:27.0:22
50325536213
0.008249206:27.0:22
50325536321
0.0049783257:136.0:12
50325536324
0.0024413536:136.0:12
50325536331
0.007878113:136.0:12
50325537332
0.005167532:136.0:12
50325537333
0.010681029:136.0:12
50325537424
0.00064694934:136.0:12


0.0015319473:0.0:1
50324585212
0.0013228151:0.0:1
50324585214
0.0046246154:0.0:1
50324585232
0.004289332:0.0:1
50324586313
0.001512771:18.0:20
50324586324
0.00032264064:18.0:20
50324586331
0.005769592:18.0:20
50324586431
0.003881681:18.0:20
50324586432
0.0013959177:18.0:20
50324595124
0.0052720215:18.0:20
50324595131
0.0016702171:17.0:1
50324595434
0.000119305914:17.0:1
50324595441
0.006838056:17.0:1
50324595442
0.0010407062:17.0:1
50324595443
0.0056360634:17.0:1
50324595444
0.00027176365:17.0:1
50324596333
0.0073084403:17.0:1
50324596341
0.0070925094:79.0:18
50324596412
0.00080602244:79.0:18
50324596414
0.0015328247:79.0:18
50324658222
0.0048906486:24.0:4
50324659131
0.008239109:24.0:4
50324660132
0.00071338285:24.0:4
50324660134
0.0016488035:24.0:4
50324661332
0.00042143092:24.0:4
50324663224
0.014792949:24.0:4
50324663234
0.00070790667:24.0:4
50324666343
0.00028305454:24.0:4
50324667412
0.0015794166:24.0:4
50324667433
0.001957656:24.0:4
50324671111
0.001577317:24.0:4
50324671114
0.0

50325556442
0.0030427412:95.0:20
50325556443
0.0008610451:95.0:20
50325557314
0.015821908:95.0:20
50325557331
0.008902255:95.0:20
50325557333
0.014324865:95.0:20
50325557334
0.00037502497:95.0:20
50325558134
0.00021481235:95.0:20
50325558141
0.0004800933:95.0:20
50325558221
0.00054545887:95.0:20
50325558311
0.009232117:95.0:20
50325558312
0.007213968:95.0:20
50325559113
0.0002988493:95.0:20
50325559121
0.0001422842:95.0:20
50325559143
5.9012324e-05:95.0:20
50325559211
0.00021326353:95.0:20
50325559212
0.007985693:95.0:20
50325565142
0.0018024134:136.0:12
50325565212
0.0006709421:136.0:12
50325565213
0.0010215587:136.0:12
50325565232
0.009582792:36.0:8
50325565243
0.001681162:36.0:8
50325565421
0.00064090407:36.0:8
50325566142
0.0059161223:36.0:8
50325566144
0.005477301:36.0:8
50325566212
0.009709053:95.0:20
50325566222
0.006949003:95.0:20
50325566223
0.0019427964:95.0:20
50325566444
0.019217927:36.0:8
50325567111
4.8873015e-05:95.0:20
50325567113
0.0009190007:95.0:20
50325567121
0.0042

50324556234
0.00052176975:27.0:3
50324556241
0.009416278:27.0:3
50324557232
0.0010680011:27.0:3
50324557241
0.0005023349:27.0:3
50324557243
0.0006060903:27.0:3
50324564114
0.00027220068:0.0:1
50324564234
0.0032312605:0.0:1
50324566342
0.0021373862:0.0:1
50324566344
0.00026744755:0.0:1
50324566411
0.00037488237:0.0:1
50324566413
0.009450058:0.0:1
50324574331
9.280746e-06:0.0:1
50324574414
0.010913763:0.0:1
50324574423
0.0015615607:0.0:1
50324575141
0.007118657:0.0:1
50324575243
0.00016900059:0.0:1
50324575441
0.009416224:0.0:1
50324575444
0.00010594679:0.0:1
50324576114
0.009136108:0.0:1
50324576121
0.0041120546:0.0:1
50324576122
0.0053135254:0.0:1
50324576123
8.785643e-05:0.0:1
50324584344
0.0033746844:0.0:1
50324584414
0.020911:0.0:1
50324584423
0.00057950197:0.0:1
50324584424
0.0054340223:0.0:1
50324585121
0.0065423325:0.0:1
50324585422
0.0019403726:18.0:20
50324585433
0.008994836:18.0:20
50324585443
0.009148927:18.0:20
50324594243
0.0009656855:17.0:1
50324594411
0.0006203084:17.0:1


...

0.004663052:38.0:12
50325614441
0.01891129:38.0:12
50325615334
0.004552785:38.0:12
50325615343
0.007527065:38.0:12
50325615431
4.527613e-05:38.0:12
50325615433
0.003606095:38.0:12
50325615441
0.005269972:142.0:7
50325620441
0.0024560997:80.0:9
50325622424
0.0014039208:104.0:12
50325622442
0.0008003018:104.0:12
50325622443
0.0024536704:104.0:12
50325622444
0.006226398:104.0:12
50325623233
0.004226286:104.0:12
50325623234
0.0010855193:104.0:12
50325623311
0.0015279641:104.0:12
50325623312
0.0022908375:104.0:12
50325623421
0.0012675265:104.0:12
50325623424
5.6466786e-05:104.0:12
50325623431
0.0022746865:104.0:12
50325623432
0.0087811425:104.0:12
50325623433
0.0071574925:104.0:12
50325624224
0.000721329:38.0:12
50325624232
0.007886943:38.0:12
50325624233
0.0006247837:104.0:12
50325624234
0.0014007669:104.0:12
50325624243
0.0005625654:38.0:12
50325624434
0.0002184913:46.0:5
50325624441
0.0001873672:104.0:12
50325624442
0.0028350079:46.0:5
50325624443
0.02876859:46.0:5
50325625112
0.00669462

0.007791782:70.0:2
50325566223
0.0009788098:87.0:15
50325566312
0.0014848565:51.0:3
50325566332
0.00345221:51.0:3
50325566333
0.024210472:70.0:2
50325566433
0.0067161117:51.0:3
50325566434
0.0004547349:51.0:3
50325566443
0.0010159783:51.0:3
50325566444
0.018234506:51.0:3
50325567111
0.00014654873:87.0:15
50325567113
0.0007038966:87.0:15
50325567121
0.004122425:87.0:14
50325567143
0.0011437361:122.0:1
50325567144
0.011176992:122.0:1
50325567214
0.01134225:87.0:15
50325567221
0.0069878018:87.0:15
50325567222
4.1549327e-05:87.0:15
50325567312
0.0014036811:51.0:3
50325567313
0.000608163:51.0:3
50325567321
0.0023417552:122.0:1
50325567322
0.005383604:122.0:3
50325567324
0.018557113:122.0:3
50325567333
0.0004581631:51.0:3
50325567343
9.606592e-05:51.0:3
50325567431
0.018144907:122.0:1
50325567442
0.010455154:122.0:3
50325567443
0.013595789:122.0:3
50325567444
0.0052556065:122.0:3
50325568111
0.0056855613:122.0:3
50325568112
0.0056220484:122.0:3
50325568113
0.012347649:122.0:3
50325568114
0.0

0.0040125363:104.0:12
50325624334
0.002036924:104.0:12
50325625142
0.0019526794:142.0:7
50325625143
0.0019494072:38.0:12
50325625214
0.003486922:142.0:7
50325625231
0.005654613:142.0:7
50325625232
0.00038707108:142.0:7
50325625234
0.0017870355:142.0:7
50325626244
0.0030852798:142.0:7
50325626321
0.0016001267:142.0:7
50325626331
0.00026123144:142.0:7
50325626414
0.010033325:142.0:7
50325628332
0.00061033666:142.0:7
50325628333
0.00058571226:142.0:7
50325629422
0.0010884185:121.0:10
50325629442
0.0013681119:121.0:10
50325630111
0.002184514:80.0:9
50325630423
0.006492309:80.0:9
50325630431
0.005265776:80.0:9
50325630442
0.0017328729:80.0:9
50325632432
0.00036193733:115.0:6
50325632433
0.0003311457:115.0:6
50325632441
0.005576052:115.0:6
50325632442
0.0013513295:115.0:6
50325632443
0.012760403:115.0:6
50325633131
0.0067272424:74.0:1
50325633132
0.015548443:46.0:5
50325633134
0.011170407:46.0:5
50325633142
0.004423823:46.0:5
50325633143
0.004438242:46.0:5
50325633324
0.0001162393:46.0:5
503

50325659334
0.0035611587:121.0:10
50325659342
0.0072582895:121.0:10
50325659411
0.021275004:121.0:10
50325659412
0.012945818:121.0:10
50325659413
0.0015414748:121.0:10
50325659414
0.0060528233:121.0:10
50325660234
0.00096259057:0.0:3
50325660323
0.0018093386:64.0:28
50325660324
0.0011070395:64.0:24
50325660333
0.016155764:64.0:24
50325660334
0.050817765:64.0:28
50325660434
0.008927156:0.0:2
50325660442
0.04721971:0.0:3
50325660444
0.01183862:0.0:2
50325661211
0.0045197196:0.0:3
50325661212
0.0056468993:0.0:2
50325661213
0.010025479:0.0:3
50325661222
0.0002732065:59.0:8
50325661223
0.0054832445:0.0:3
50325661324
0.0025241468:0.0:3
50325661331
0.01455719:0.0:2
50325661332
0.003641749:0.0:2
50325661333
0.008490739:0.0:2
50325661334
0.002983673:0.0:3
50325661341
0.003613254:0.0:3
50325661434
0.0014748042:0.0:3
50325661441
0.010401528:0.0:2
50325661442
0.001693368:0.0:3
50325661443
9.689294e-05:0.0:3
50325661444
0.0034652688:0.0:2
50325662111
0.000106400345:59.0:8
50325662224
0.012021885:46

50325603111
0.0364442:113.0:0
50325603112
0.0013605068:113.0:0
50325603122
0.0014931981:113.0:0
50325603123
0.0073736296:113.0:0
50325603124
0.00025048375:113.0:0
50325603313
0.0009441837:113.0:0
50325603321
0.002187007:113.0:0
50325603322
0.0006668244:113.0:0
50325603323
0.004378412:113.0:0
50325603324
0.007577697:113.0:0
50325605143
0.0002363467:113.0:0
50325605214
0.0031157215:113.0:0
50325605232
0.0007737647:113.0:0
50325605233
0.0033393644:113.0:0
50325605234
0.00028002472:113.0:0
50325605322
0.0010809111:113.0:0
50325606133
0.005468702:113.0:0
50325611422
0.0003648111:31.0:6
50325611424
0.001064053:31.0:6
50325612121
0.0017722053:31.0:6
50325612221
0.0016776007:31.0:6
50325613311
0.012983234:31.0:6
50325613313
0.0030158064:31.0:6
50325613323
0.0012282506:38.0:12
50325613334
0.0030700788:38.0:12
50325615214
0.0057759196:113.0:0
50325615221
0.0015035095:113.0:0
50325615233
0.00097561954:38.0:12
50325615324
0.0036014963:38.0:12
50325615331
0.00042599387:38.0:12
50325620321
0.0009812

In [ ]:
#ConvertMeshtoRoad

In [ ]:
connection.close()